In [ ]:
import os
import cv2
import numpy as np

# video
import dxcam
import win32gui

# model
import torch
from ultralytics import YOLO

from IPython.display import display, Image, clear_output
import ipywidgets as widgets
import time

In [ ]:
# paths
NOTEBOOK_DIR = os.getcwd()
ROOT = os.path.join(NOTEBOOK_DIR, os.path.pardir)
CHECKPOINTS_DIR = os.path.join(ROOT, "checkpoints")
YOLO_CHECKPOINT_PATH = os.path.join(CHECKPOINTS_DIR, "yolo11s-det.pt")

# torch cfg
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# misc
WINDOW_NAME = "Liftoff"

In [ ]:
def get_window_rect(window_name):
    window = win32gui.FindWindow(None, window_name)
    if not window:
        raise Exception(f"Window '{window_name}' not found")
    left, top, right, bot = win32gui.GetWindowRect(window)
    return (left, top, right, bot)

def get_game_region(window_name):
    # find and correct game region
    region = get_window_rect(window_name)
    x_top, y_top, x_bot, y_bot = region
    x_top += 15
    y_top += 15
    x_bot -= 15
    y_bot -= 15
    region = (x_top, y_top, x_bot, y_bot)

    # check if region out of bounds
    if any(pos < 0 for pos in region):
        raise Exception("Window not found")

    return region

def capture_game(camera):
    # camera.start()
    frame = camera.get_latest_frame()
    if frame is not None:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame
    
    raise Exception("Window not found")

In [ ]:
# init model
model = YOLO(YOLO_CHECKPOINT_PATH).to(DEVICE)

In [ ]:
# start game capturing
region = get_game_region(WINDOW_NAME)
camera = dxcam.create(region=region)
camera.start()

out = widgets.Output()
display(out)

while True:
    frame = capture_game(camera)

    if frame is None:
        break

    results = model.predict(frame, conf=0.5, verbose=False)
    annotated_frame = results[0].plot()

    _, jpeg = cv2.imencode('.jpeg', annotated_frame)
    with out:
        clear_output(wait=True)
        display(Image(data=jpeg.tobytes()))
        time.sleep(0.01)